# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
weather_df = pd.read_csv(r"C:\Users\mseid\UCBWork\python_api_challenge\output_data\weather_data.csv")
weather_df.head()

,City,Country,Latitude,Longitude,Temperature (F),Cloudiness (%),Humidity (%),Wind Speed (MPH)
0,Arman',RU,59.70,150.17,17.20,50,90,2.24
1,Nantucket,US,41.28,-70.10,47.57,90,87,16.11
2,Salento,CO,4.64,-75.57,78.80,40,69,5.82
3,Micheweni,TZ,-4.97,39.83,85.03,28,70,10.63
4,Villa Carlos Paz,AR,-31.42,-64.50,67.95,28,37,11.41


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
gmaps.configure(api_key=g_key)
coords = weather_df[["Latitude", "Longitude"]]
humidity = weather_df["Humidity (%)"].astype(float)

In [16]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(coords, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_df = weather_df.loc[weather_df["Temperature (F)"] < 80, :]
ideal_df = ideal_df.loc[ideal_df["Temperature (F)"] > 70, :]
ideal_df = ideal_df.loc[ideal_df["Wind Speed (MPH)"] < 10, :]
ideal_df = ideal_df.loc[ideal_df["Cloudiness (%)"] == 0, :]
ideal_df

,Unnamed: 0,City,Country,Latitude,Longitude,Temperature (F),Cloudiness (%),Humidity (%),Wind Speed (MPH)
214,214,Vīsāvadar,IN,21.38,70.68,74.64,0,69,6.33
226,226,Kalamb,IN,19.05,73.95,77.04,0,32,7.58
328,328,Caborca,MX,30.58,-111.10,71.35,0,56,5.99
358,358,Cockburn Town,TC,21.46,-71.14,76.89,0,63,8.95


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_df[["City", "Country", "Latitude", "Longitude"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df  

,City,Country,Latitude,Longitude,Hotel Name
214,Vīsāvadar,IN,21.38,70.68,
226,Kalamb,IN,19.05,73.95,
328,Caborca,MX,30.58,-111.10,
358,Cockburn Town,TC,21.46,-71.14,


In [13]:
for index,row in hotel_df.iterrows():

    params = {"radius": 5000,
            "type": "lodging",
            "key": g_key,
            "location": f"{row['Latitude']}, {row['Longitude']}"}
       
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response = requests.get(base_url, params=params)
    response = response.json()
    
    hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]

hotel_df

,City,Country,Latitude,Longitude,Hotel Name
214,Vīsāvadar,IN,21.38,70.68,Brahmanad Dham Guest House
226,Kalamb,IN,19.05,73.95,Hotel Ravikiran
328,Caborca,MX,30.58,-111.10,Hotel Los Lagos
358,Cockburn Town,TC,21.46,-71.14,Osprey Beach Hotel


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [15]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))